<a href="https://colab.research.google.com/github/gd1m3y/Sentiment-Prediction-Using-Elmo/blob/master/sentiment_classification_using_elmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import spacy
import tensorflow as tf
import tensorflow_hub as hub
# tf.compat.v1.enable_eager_execution()


In [3]:
import pickle 
import pandas as pd
import re 
import numpy as np
import time

In [4]:
#reading the data 

train = pd.read_csv('/content/train_2kmZucJ.csv')
test = pd.read_csv('/content/test_oJQbWVk.csv')




In [5]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [6]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x : re.sub(r'http\S+','',x))

test['clean_tweet'] = test['tweet'].apply(lambda x : re.sub(r'http\S+','',x))


In [7]:
train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,#fingerprint #Pregnancy Test #android #apps #...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...,What amazing service! Apple won't even talk to...


In [8]:
import string

In [9]:
#basic cleaning
punc = string.punctuation
train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in punc))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in punc))

#convert text to lowercase

train['clean_tweet'] = train['clean_tweet'].str.lower()

test['clean_tweet'] = test['clean_tweet'].str.lower()

#remove numbers

train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]"," ")

test['clean_tweet'] = test['clean_tweet'].str.replace('[0-9]'," ")

#remove whitespaces

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))


In [10]:
train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps beauti...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks to m...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememories un...
3,4,0,I'm wired I know I'm George I was made that wa...,im wired i know im george i was made that way ...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple wont even talk to m...


In [11]:
#normalization

nlp = spacy.load('en',disable=['parser','ner'])

#function to lemmatize text

def lemmatization(texts):
  output = []
  for i in texts:
    s = [token.lemma_ for token in nlp(i)]
    output.append(' '.join(s))
  return output

In [12]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [13]:
train.head()

,id,label,tweet,clean_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps beauti...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thank to -P...
2,3,0,We love this! Would you go? #talk #makememorie...,-PRON- love this would -PRON- go talk makememo...
3,4,0,I'm wired I know I'm George I was made that wa...,-PRON- be wire i know -PRON- be george i be ma...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple will not even talk ...


In [23]:
#tf hub module
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [60]:
# just a random sentence
x = ["i really hate you and i really hate the way you talk"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  # return average of ELMo features
  k = sess.run(tf.reduce_mean(embeddings,1))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [61]:
print(k)
lreg.predict(k)

[[-0.28116867 -0.32125401 -0.1241304  ...  0.19192763  0.5303723
   0.01820945]]


array([1])

In [43]:

def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [14]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [39]:
elmo_train_new.shape

(7920, 1024)

In [15]:

from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [51]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression(max_iter=150)
lreg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=150,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
preds_valid = lreg.predict(xvalid)

In [53]:
f1_score(yvalid, preds_valid)

0.7833333333333333

In [54]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [57]:
test.clean_tweet[0]

'i hate the new iphone upgrade will not let -PRON- download app ugh apple suck'

In [55]:
preds_test

array([1, 1, 1, ..., 0, 1, 0])

In [ ]:
# from google.colab import files
# files.download('/content/elmo_train_03032019.pickle') 



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Testing Out Our Model**

In [89]:
def create_embedding(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  # embeddings.shape
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

def lemmatization(texts):
  output = []
  for i in nlp(texts):
    output.append(i.lemma_)
  output = " ".join(i for i in output)
  return output

def preprocess(x):
  #removing  any links present
  x = re.sub(r'http\S+','',x)
  #removing any punctuation
  x = "".join(i for i in x if i not in punc)
  #removing any numbers 
  x = x.replace('[0-9]'," ")
  #converting all to lower case
  x = x.lower()
  #removing extra white spaces
  x = " ".join(i for i in x.split())

  #lemmaziation / normalization of text 
  x = lemmatization(x)
  return[x]
sentence = input()

EMB = create_embedding( preprocess(sentence if sentence != None else train.tweet[0]))
# print(EMB)
print('Positive' if lreg.predict(EMB)[0] == 0 else 'Negative')
# def test(x):



hello
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Positive
